# fDOM Data Merging
This file's main goal is to merge all labeled timeline data into a single source to allow for easier data augmentation, as well a classifier that can detect all types of anomalies in one, rather than one classifier for each one. This specific file merges fDOM labeled data into a single file.

In [ ]:
# Imports
import csv
import pandas as pd

The following functions are helpers for the rest of the merging process.

In [ ]:
def load_labeled_dataset(filename):
    """ Read in labeled data and return the read in data """
    with open(filename, 'r', newline='') as f:
        reader = csv.reader(f, delimiter=',')
        next(reader)
        truths = [[float(row[0]), float(row[1]), row[2], int(row[3])] for row in reader] 
        f.close()  

    return truths 

def print_entire_df(dataframe):
    """Print out the entire contents of a dataframe, useful if you need to see differences (WARNING: ENTIRE OUTPUT WILL GO INTO GITHUB IF FILE COMMITTED WITH CELL OUTPUT"""
    with pd.option_context('display.max_rows', None, 'display.max_columns', None):
        print(dataframe)

## Load in data

In [ ]:
# Load in all of the datasets:
fDOM_PLP_path = '../Data/labeled_data/ground_truths/fDOM/fDOM_PLP/julian_time/fDOM_PLP_0k-300k.csv'
fDOM_SKP_path = '../Data/labeled_data/ground_truths/fDOM/fDOM_SKP/julian_time/fDOM_SKP_0k-300k.csv'
fDOM_PP_path = '../Data/labeled_data/ground_truths/fDOM/fDOM_PP/julian_time/fDOM_PP_0k-300k.csv'

# load in dataset from original function (MIGHT BE UNNECESSARY)
fDOM_PLP_truths = load_labeled_dataset(fDOM_PLP_path)
fDOM_SKP_truths = load_labeled_dataset(fDOM_SKP_path)
fDOM_PP_truths = load_labeled_dataset(fDOM_PP_path)

# Load in dataframes
fDOM_PLP_df = pd.read_csv(fDOM_PLP_path)
fDOM_SKP_df = pd.read_csv(fDOM_SKP_path)
fDOM_PP_df = pd.read_csv(fDOM_PP_path)

# update indices to use timestamp
fDOM_PLP_df.set_index('timestamp_of_peak', inplace=True)
fDOM_SKP_df.set_index('timestamp_of_peak', inplace=True)
fDOM_PP_df.set_index('timestamp_of_peak', inplace=True)

In [ ]:
# Visualize these dataframes
print("PLP Head:")
print(fDOM_PLP_df.head())

print("\nSKP Head:")
print(fDOM_SKP_df.head())

print("\nPP Head:")
print(fDOM_PP_df.head())

## Peak Precendence
The following code block sets the order precendence of peaks. 

In [ ]:
# SET PRECENDENCE OF PEAKS
# skyrocketing <- phantom <- plummeting
TOP = fDOM_SKP_df
TOP_ACRO = "SKP"
TOP_NO_ACRO = "NSKP"

SECOND = fDOM_PP_df
SECOND_ACRO = "PP"
SECOND_NO_ACRO = "NPP"

THIRD = fDOM_PLP_df
THIRD_ACRO = "PLP"
THIRD_NO_ACRO = "NPLP"


## Merge Data
We concat all values into a single dataframe, stable sort by timestamp, and then drop all duplicates.
Following this, we then rename all labels starting with N to be NAP (not anomaly peaks)

Using the stable sorting method keeps the indices in the correct order, so when we drop duplicates our peak precendence is saved should there be any overlapping ones.
Following t

In [ ]:
# concat all three dataframes
df = pd.concat([TOP, SECOND, THIRD])

# sort values
df = df.sort_values(by=['timestamp_of_peak'], kind='stable')

# remove duplicates
df = df[~df.index.duplicated(keep='first')]

# rename all labels that start with N to be NAP using regex
final_data = df.replace(to_replace='N(.*)', value="NAP", regex=True)


## Output to CSV
The following codeblock exports the newly created timeline.

In [ ]:
# add code here...